| ![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-smaller.PNG)  | [National Science Data Fabric](https://nationalsciencedatafabric.org/) [Jupyter notebook](https://jupyter.org/) <br> created by  [Valerio Pascucci](http://cedmav.com/) and  [Giorgio Scorzelli](https://www.sci.utah.edu/people/scrgiorgio.html)  |  
|---|:---:|


# IMPORTANT, READ CAREFULLY

## this notebook does NOT work in Chrome, please use Firefox

In [ ]:
import os,sys,logging,time
import numpy as np

print(sys.executable)

os.environ["BOKEH_ALLOW_WS_ORIGIN"]="*"
os.environ["BOKEH_LOG_LEVEL"]="debug"
os.environ["VISUS_CPP_VERBOSE"]="0"
os.environ["VISUS_NETSERVICE_VERBOSE"]="0"
os.environ["VISUS_VERBOSE_DISKACCESS"]="0"

import panel as pn
pn.extension(log_level="DEBUG",notifications=True, sizing_mode="stretch_width")

if True:
    sys.path.append("c:/projects/openvisus/build/RelWithDebInfo")
    sys.path.append("c:/projects/openvisuspy/src")

from openvisuspy import Slice, SetupJupyterLogger, LoadDataset, ExecuteBoxQuery
logger=SetupJupyterLogger(logging_level=logging.DEBUG) 
print(time.time(),"OpenVisus imported")

#  <center> This is a preview of the data:
![Visualization of Covid-19 cases](https://www.sci.utah.edu/~pascucci/public/DRP-preview.gif)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# VTK is needed for VTKVolumePlot 
# see https://panel.holoviz.org/reference/panes/VTKVolume.html
pn.extension('vtk') 

def MyApp(url, endh=21, height=800):
    
    db=LoadDataset(url)
    print(f"Loaded dataset")
    print(f"  fields={db.getFields()}")
    print(f"  logic_box={db.getLogicBox()}")
    W,H,D=db.getLogicBox()[1]
    
    access=db.createAccess()
    data=list(ExecuteBoxQuery(db, access=access, endh=endh,timestep=0,num_refinements=1))[0]['data']
    print(f"Got data shape={data.shape} dtype={data.dtype}")

     # generate a panel for numpy data
    volume = pn.panel(
        data, 
        sizing_mode='stretch_both',
        orientation_widget =True,
        display_slices=True,
        spacing = (1, 1, 1),
        controller_expanded=False)

    # other widgets, with callbacks
    experiment  = pn.widgets.IntSlider(name='Experiment', start=0, end=1, step=1, value=0 )
    @pn.depends(experiment)
    def experiment_callback(value):
        nonlocal volume
        data=list(ExecuteBoxQuery(db, access=access, endh=endh,timestep=value,num_refinements=1))[0]['data']
        volume.object = data
        return " "    
    
    slider_i = pn.widgets.IntSlider(name='i-slice', start=0, end=data.shape[0], value = data.shape[0]//2 )
    @pn.depends(slider_i)
    def slider_i_callback(value):
        nonlocal volume
        volume.slice_i = value 
        return " "    
    
    slider_j = pn.widgets.IntSlider(name='j-slice', start=0, end=data.shape[1], value = data.shape[1]//2 )
    @pn.depends(slider_j)
    def slider_j_callback(value):
        nonlocal volume
        volume.slice_j = value 
        return " "    
    
    slider_k = pn.widgets.IntSlider(name='k-slice', start=0, end=data.shape[2], value = data.shape[2]//2 )
    @pn.depends(slider_k)
    def slider_k_callback(value):
        nonlocal volume
        volume.slice_k = value 
        return " "    
    
    show_volume = pn.widgets.Checkbox(name='Show Volume',value=True)
    @pn.depends(show_volume)
    def show_volume_callback(value):
        nonlocal volume
        volume.display_volume = value 
        return " "    
    
    show_slices = pn.widgets.Checkbox(name='Show Slices',value=True)
    @pn.depends(show_slices)
    def show_slices_calback(value):
        nonlocal volume
        volume.display_slices = value 
        return " "    

    main_layout=pn.Column(
        experiment,
        slider_i,
        slider_j,
        slider_k,
        pn.Row(
            show_volume,
            show_slices
        ),
        volume, 
        experiment_callback,
        slider_i_callback,
        slider_j_callback,
        slider_k_callback,
        show_volume_callback,
        show_slices_calback,
        height = height,
        css_classes=['panel-widget-box'],
        sizing_mode='stretch_width',
        width_policy='max')

    return main_layout

MyApp('http://atlantis.sci.utah.edu:80/mod_visus?dataset=fly_scan_time-s-midx/data&cached=idx', endh=21, height=800)

#  <center> This is a preview of the data
![Visualization of Covid-19 cases](https://www.sci.utah.edu/~pascucci/public/CHESS-visus_recon_combined_1_fullres_zip.gif)

In [ ]:
MyApp(url='http://atlantis.sci.utah.edu:80/mod_visus?dataset=chess-zip&cached=1', endh=21, height=800)

#  <center> This is a preview of the data:
![Visualization of Covid-19 cases](https://www.sci.utah.edu/~pascucci/public/CHESS-visus_recon_combined_1_2_3_fullres_zip.gif)

In [ ]:
MyApp(url='https://atlantis.sci.utah.edu/mod_visus?dataset=chess-recon_combined_1_2_3_fullres_zip&cached=1', endh=21, height=800)